In [10]:
# Transformer >= 3.0.0 이상부터
# BertTokenizer.toknizer()사용가능 

# vizualization library
import matplotlib.pyplot as plt
import numpy as np

# pytorch library
import torch # the main pytorch library
import torch.nn.functional as f # the sub-library containing different functions for manipulating with tensors

# huggingface's transformers library
from transformers import BertModel, BertTokenizer

import math
import random
import os

device = torch.device("cpu" if torch.cuda.is_available() else "cuda")
print(device)

cpu


In [2]:
bert_version = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(bert_version)
model = BertModel.from_pretrained(bert_version)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
model = model.eval()
model = model.to(device)

In [4]:
def visualize(distances, figsize=(20, 10), titles=None):
    # get the number of columns
    ncols = len(distances)
    # create the subplot placeholders
    fig, ax = plt.subplots(ncols=ncols, figsize=figsize)
    
    for i in range(ncols):
        
        # get the axis in which we will draw the matrix
        axes = ax[i] if ncols > 1 else ax
        
        # get the i-th distance
        distance = distances[i]
        
        # create the heatmap
        axes.imshow(distance)
        
        # show the ticks
        axes.set_xticks(np.arange(distance.shape[0]))
        axes.set_yticks(np.arange(distance.shape[1]))
        
        # set the tick labels
        axes.set_xticklabels(np.arange(distance.shape[0]))
        axes.set_yticklabels(np.arange(distance.shape[1]))
        
        # set the values in the heatmap
        for j in range(distance.shape[0]):
            for k in range(distance.shape[1]):
                text = axes.text(k, j, str(round(distance[j, k], 3)),
                               ha="center", va="center", color="w")
        
        # set the title of the subplot
        title = titles[i] if titles and len(titles) > i else "Text Distance"
        axes.set_title(title, fontsize="x-large")
        
    fig.tight_layout()
    plt.show()
    
def load_data(_path):
    datas = [] 
    with open(_path, 'r') as f:
        lines = f.readlines()
        for line in lines[:10]:
            datas.append(line.strip())
    return datas

def load_data_at_EM(memory):
    datas = [] 
    for tmp in memory:
        datas.append(tmp['text'])
    return datas

In [6]:
##### CONFIG AREA
THRESHOLD = 0.98 # 0.85
THRESHOLD_F_TO_LONGTERM = 2 # F: ? 이상은 Longterm으로 전이시킬 후보
SHORTTERM_REPLAY_RATIO = 0.2 # 현재도메인의 20%개수만을 가져옴 
data_template = {
    'T' : 0,  # 들어오고나서 지난횟수 
    'F' : 0,  # cos simil의 max값이 > Threshold한 횟수 
    'D' : '', # Domain label
    'text' : ''
}

##### MEM retention var
freq_0_retention_timestep = 1 # F:0의 데이터들은 T:1까지 살려둠(1회반복)
freq_1_retention_timestep = 3 # F:1의 데이터들은 T:3까지 살려둠(3회반복)

##### Domain data loader 
DATA_DIR = './multiwoz2_data/1008_multiline_process/'
# ARPER RUM1: 0 5 2 1 3 4 
DOMAIN_SEQ = ['0_attraction', '5_train', '2_hotel', '1_booking', '3_restaurant', '4_taxi', ]
data_dict = {}

for domain in DOMAIN_SEQ:
    datas = load_data(DATA_DIR+"{}_train.txt".format(domain,))
    data_dict[domain] = datas
    print(">>> {} domain {} lines loads".format(domain, len(datas)))

>>> 0_attraction domain 10 lines loads
>>> 1_booking domain 10 lines loads
>>> 2_hotel domain 10 lines loads
>>> 3_restaurant domain 10 lines loads
>>> 4_taxi domain 10 lines loads
>>> 5_train domain 10 lines loads


In [12]:
##### Make train data pool
train_data_pool = {}  # {'domain' : [Q|A, Q|A]} 형태 
episodic_memory = []  # {'[{template}, {template}]} 형태
long_term_pool = [] # Long term에 넘겨질 Pool [Q|A, Q|A] 형태

for domain_idx, domain in enumerate(DOMAIN_SEQ):
    print(">>> Make [{}] domain train data".format(domain))
    total_EM_data_num = len(episodic_memory)
    print("\t Now Episodic Mem : {} ".format(total_EM_data_num))
    
    
    if episodic_memory == []:# 처음 
        train_data_pool[domain] = data_dict[domain].copy() # 처음 도메인 그대로 사용
        # EM에 데이터 저장
        for data in data_dict[domain]:
            episodic_memory.append({
                'T' : 0,
                'F' : 0,
                'D' : domain,
                'text' : data
            })
    else: # EM에 데이터가 쌓여있는 단계 
        """ TODO
        1. EM안의 데이터 <-> 새롭게 들어오는데이터 간의
           cosine sim값을 계산하여 > Threshold값의 
           EM안의 데이터들은 F를 +1해준다
        
        2. EM policy에 따라 longterm에 넘길꺼, 삭제할거,
           train_data_pool에 다시 줄거를 구분 
        
        3. EM의 Timestep + 1 
        
        4. 현재도메인데이터 EM에 추가
        
        
    
        """
        ## TODO1 : EM이랑 새로운도메인 COSINE SIMIL값 계산 
        prev_data_idx = total_EM_data_num
    
        current_domain_data = data_dict[domain].copy() # [Q|A, Q|A]
        print("\t Current domain data : ", len(data_dict[domain]))
        memory_data = load_data_at_EM(episodic_memory).copy()
        assert total_EM_data_num==len(memory_data)
        #print(memory_data)
        
        # 기존 + 현재들어오는걸 합쳐서 토크나이징 
        texts = memory_data + current_domain_data
        
        encodings = tokenizer(
            texts, # the texts to be tokenized
            padding=True, # pad the texts to the maximum length (so that all outputs have the same length)
            return_tensors='pt' # return the tensors (not lists)
        )
        encodings = encodings.to(device)
        
        # disable gradient calculations
        # 임베딩 실시
        with torch.no_grad():
            embeds = model(**encodings)
        embeds = embeds[0]
        
        # CLS 토큰들 가지고 Cosin simil 계산
        CLSs = embeds[:, 0, :]
        
        # normalize the CLS token embeddings
        normalized = f.normalize(CLSs, p=2, dim=1)
        # calculate the cosine similarity
        cls_dist = normalized.matmul(normalized.T)
        cls_dist = cls_dist.numpy()
        
        #visualize([cls_dist], titles=["CLS"])
        em_max_cos_simil = [0 for _ in range(total_EM_data_num) ] 
        for row in range(total_EM_data_num):
            em_max_cos_simil[row] = np.max(cls_dist[row][total_EM_data_num:])
            
        # TRESHOLD가 넘은거 찾아서 F ++ 해준다 
        #print(em_max_cos_simil)
        for idx, simil in enumerate(em_max_cos_simil):
            if simil > THRESHOLD:
                target_data = memory_data[idx]
                for tmp in episodic_memory:
                    if tmp['text'].strip() == target_data.strip():
                        tmp['F'] += 1 
        
        ## TODO2 : EM 정책에 따라 longterm, 삭제할거, train_pool에 줄거 분류
        # Longterm에 넘겨질거 
        update_episodic_memory = [] 
        for data in episodic_memory:
            if data['F'] >= 2:
                pass # Long term 메모리로 보내서 작업할 데이터들 
            else:
                update_episodic_memory.append(data)
        print("\t Long term으로 전이될 수 : ", len(episodic_memory)-len(update_episodic_memory))
        episodic_memory = update_episodic_memory.copy() # update 
        
        # 삭제할거 
        #freq_0_retention_timestep = 1 # F:0의 데이터들은 T:1까지 살려둠
        #freq_1_retention_timestep = 3 # F:1의 데이터들은 T:3까지 살려둠
        update_episodic_memory = []
        for data in episodic_memory:
            if data['F'] == 0:
                if data['T'] >= freq_0_retention_timestep:
                    pass # 삭제할거
                else:
                    update_episodic_memory.append(data)
                    
            elif data['F'] == 1:
                if data['T'] >= freq_1_retention_timestep:
                    pass # 삭제할거 
                else:
                    update_episodic_memory.append(data)
        
        print("\t EM에서 제거 된 수 : ", len(episodic_memory)-len(update_episodic_memory))
        episodic_memory = update_episodic_memory.copy() # update
        
        # train_pool에 현재도메인 + EM메모리 추가 
        # !! 여기서 개수를 조정할수도 있음 현재는 20퍼
        # 넘겨주는건 각도메인마다 현재데이터*20퍼 / 도메인개수 
        replay_data_list = [] 
        
        replay_data_num = math.ceil(len(current_domain_data) * SHORTTERM_REPLAY_RATIO)
        per_domain_replay_num = math.ceil(replay_data_num / (domain_idx + 1))
        print("\t Replay할 데이터 총 수 : ", replay_data_num)
        print("\t 도메인당 Replay 데이터 수 : ", per_domain_replay_num)
        
        # 도메인별로 정리 
        domain_data_dict = {}
        for data in episodic_memory:
            try:
                domain_data_dict[data['D']].append(data['text'])
            except:
                domain_data_dict[data['D']] = [data['text']]
        
        # 도메인별 돌면서 뽑기 
        for dom_k, dom_v in domain_data_dict.items():
            replay_data_list += random.sample(dom_v, per_domain_replay_num)
        
        print("\t 실제 Replay 준비된 데이터 수 : ", len(replay_data_list))
        
        
        train_data_pool[domain] = current_domain_data.copy()
        for replay_data in replay_data_list:
            train_data_pool[domain].append(replay_data)
            
        ## TODO3: Timestep + 1 
        for data in episodic_memory:
            data['T'] += 1 
        
        ## TODO4: 현재도메인 데이터 EM에 추가
        for data in data_dict[domain]:
            episodic_memory.append({
                'T' : 0,
                'F' : 0,
                'D' : domain,
                'text' : data
            })
        
print(">>> DOMAIN SEQ", DOMAIN_SEQ)
for dom_k in DOMAIN_SEQ:
    print("\t {} domain... {} data".format(dom_k, len(train_data_pool[dom_k])))

#print(train_data_pool)
#print(len(train_data_pool['0_attraction']))
#print(len(train_data_pool['1_booking']))
#print(len(train_data_pool['2_hotel']))


# 끝으로 원하는 Domain_seq별 데이터들 저장해주기 
folder_name = "./multiwoz2_data/"
for domain in DOMAIN_SEQ:
    folder_name += domain[0]
folder_name = folder_name + "_data/"    
os.makedirs(folder_name, exist_ok=True)

for dom_k, dom_v in train_data_pool.items():
    with open(folder_name + "{}_train.txt".format(dom_k), 'w') as file:
        for data in dom_v:
            file.write("{}\n".format(data))
print(">>> 저장완료")


>>> Make [0_attraction] domain train data
	 Now Episodic Mem : 0 
>>> Make [1_booking] domain train data
	 Now Episodic Mem : 10 
	 Current domain data :  10
	 Long term으로 전이될 수 :  0
	 EM에서 제거 된 수 :  0
	 Replay할 데이터 총 수 :  2
	 도메인당 Replay 데이터 수 :  1
	 실제 Replay 준비된 데이터 수 :  1
>>> Make [2_hotel] domain train data
	 Now Episodic Mem : 20 
	 Current domain data :  10
	 Long term으로 전이될 수 :  4
	 EM에서 제거 된 수 :  3
	 Replay할 데이터 총 수 :  2
	 도메인당 Replay 데이터 수 :  1
	 실제 Replay 준비된 데이터 수 :  2
>>> Make [3_restaurant] domain train data
	 Now Episodic Mem : 23 
	 Current domain data :  10
	 Long term으로 전이될 수 :  4
	 EM에서 제거 된 수 :  3
	 Replay할 데이터 총 수 :  2
	 도메인당 Replay 데이터 수 :  1
	 실제 Replay 준비된 데이터 수 :  2
>>> Make [4_taxi] domain train data
	 Now Episodic Mem : 26 
	 Current domain data :  10
	 Long term으로 전이될 수 :  2
	 EM에서 제거 된 수 :  7
	 Replay할 데이터 총 수 :  2
	 도메인당 Replay 데이터 수 :  1
	 실제 Replay 준비된 데이터 수 :  3
>>> Make [5_train] domain train data
	 Now Episodic Mem : 27 
	 Current domain data :  10
	 